In [3]:
%load_ext autoreload
%autoreload 2
import netCDF4
import xarray as xr
import numpy as np
from pathlib import Path
from hmpai.pytorch.models import *
from hmpai.training import split_data_on_participants
from hmpai.pytorch.training import train, validate, calculate_class_weights, train_and_test, k_fold_cross_validate, test
from hmpai.pytorch.generators import SAT1Dataset
from hmpai.data import SAT1_STAGES_ACCURACY, SAT_CLASSES_ACCURACY
from hmpai.visualization import plot_confusion_matrix
from hmpai.normalization import *
from torchinfo import summary
from torchvision.transforms import Compose
from hmpai.pytorch.transforms import *
from hmpai.utilities import print_results, CHANNELS_2D, AR_SAT1_CHANNELS
from torch.utils.data import DataLoader
from mne.io import read_info
from pathlib import Path
from hmpai.pytorch.models import *
from hmpai.training import split_data_on_participants
import pandas as pd
from hmpai.pytorch.utilities import DEVICE, set_global_seed, get_summary_str, save_model, load_model

import os
DATA_PATH = Path(os.getenv("DATA_PATH"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
set_global_seed(42)
data_path_sat2 = DATA_PATH / "sat2/stage_data_100hz.nc"
dataset_sat2 = xr.open_dataset(data_path_sat2)
# data_path_sat1 = DATA_PATH / "sat1/stage_data_100hz.nc"
# dataset_sat1 = xr.load_dataset(data_path_sat1)
# dataset_sat1 = dataset_sat1.rename_vars({'RT': 'rt'})

dataset_sat2 = dataset_sat2.where(dataset_sat2.event_name.str.contains("accuracy"), drop=True)
# dataset_sp = dataset_sat1.where(dataset_sat1.event_name.str.contains("speed"), drop=True)

In [3]:
# shape_topological = False
# info_to_keep = ['rt']
# train_data_sat2, val_data_sat2, test_data_sat2 = split_data_on_participants(
#     dataset_sat2, 60, norm_min1_to_1
# )
# train_data_sat1, val_data_sat1, test_data_sat1 = split_data_on_participants(
#     dataset_sat1, 60, norm_min1_to_1
# )
# train_data = xr.merge([train_data_sat2, train_data_sat1])
# val_data = xr.merge([val_data_sat2, val_data_sat1])
# test_data = xr.merge([test_data_sat2, test_data_sat1])
# train_dataset = SAT1Dataset(train_data, shape_topological=shape_topological, labels=SAT_CLASSES_ACCURACY, set_to_zero=True, info_to_keep=info_to_keep, order_by_rt=False)
# val_dataset = SAT1Dataset(val_data, shape_topological=shape_topological, labels=SAT_CLASSES_ACCURACY, set_to_zero=True, info_to_keep=info_to_keep, order_by_rt=False)
# test_dataset = SAT1Dataset(test_data, shape_topological=shape_topological, labels=SAT_CLASSES_ACCURACY, set_to_zero=True, info_to_keep=info_to_keep, order_by_rt=False)

In [5]:
shape_topological = False
info_to_keep = ["rt"]
train_data_sat2, val_data_sat2, test_data_sat2 = split_data_on_participants(
    dataset_sat2, 60, norm_min1_to_1
)
# train_data_sat2 = train_data_sat2.where(train_data_sat2.labels != 0, drop=True)
# val_data_sat2 = val_data_sat2.where(val_data_sat2.labels != 0, drop=True)
# test_data_sat2 = test_data_sat2.where(test_data_sat2.labels != 0, drop=True)
# train_data_sat1, val_data_sat1, test_data_sat1 = split_data_on_participants(
#     dataset_sat1, 60, norm_min1_to_1
# )
# train_data = xr.merge([train_data_sat2, train_data_sat1])
# val_data = xr.merge([val_data_sat2, val_data_sat1])
# test_data = xr.merge([test_data_sat2, test_data_sat1])
train_dataset = SAT1Dataset(
    train_data_sat2,
    shape_topological=shape_topological,
    labels=SAT_CLASSES_ACCURACY,
    set_to_zero=True,
    info_to_keep=info_to_keep,
    order_by_rt=False,
    transform=Compose([RandomCropTransform(), ChannelsDropout(0.5, 0.4)]),
)
val_dataset = SAT1Dataset(
    val_data_sat2,
    shape_topological=shape_topological,
    labels=SAT_CLASSES_ACCURACY,
    set_to_zero=True,
    info_to_keep=info_to_keep,
    order_by_rt=False,
    # transform=Compose([RandomCropTransform()]),
)
test_dataset = SAT1Dataset(
    test_data_sat2,
    shape_topological=shape_topological,
    labels=SAT_CLASSES_ACCURACY,
    set_to_zero=True,
    info_to_keep=info_to_keep,
    order_by_rt=False,
    # transform=Compose([RandomCropTransform()]),
)

['S13', 'S10', 'S1', 'S9', 'S6', 'S12', 'S3', 'S11']


In [6]:
# Train model
model = MambaModel(embed_dim=256, n_channels=30, n_classes=5, n_layers=6, global_pool=False)
train_and_test(
    model,
    train_dataset,
    test_dataset,
    val_dataset,
    logs_path=Path("../logs/"),
    workers=0,
    batch_size=64,
    labels=SAT_CLASSES_ACCURACY,
    epochs=100,
    # epochs=1,
    weight_decay=0.001,
    label_smoothing=0.1,
    lr=0.00001
)

  0%|          | 0/153 [00:00<?, ? batch/s]

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       1.00      0.98      0.99    364925
           1       0.00      0.00      0.00     41315
           2       0.30      0.00      0.00    113216
           3       0.05      0.00      0.00    110763
           4       0.06      1.00      0.12     18745

    accuracy                           0.58    648964
   macro avg       0.28      0.40      0.22    648964
weighted avg       0.63      0.58      0.56    648964



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


  0%|          | 0/153 [00:00<?, ? batch/s]

...

  0%|          | 0/153 [00:00<?, ? batch/s]

              precision    recall  f1-score   support

           0       1.00      0.98      0.99    378509
           1       0.60      0.09      0.16     41315
           2       0.57      0.13      0.22    113216
           3       0.00      0.00      0.00    110763
           4       0.07      0.97      0.13     18745

    accuracy                           0.62    662548
   macro avg       0.45      0.43      0.30    662548
weighted avg       0.71      0.62      0.62    662548



  0%|          | 0/153 [00:00<?, ? batch/s]

              precision    recall  f1-score   support

           0       1.00      0.98      0.99    371520
           1       0.61      0.15      0.24     41315
           2       0.57      0.15      0.24    113216
           3       0.26      0.00      0.00    110763
           4       0.07      0.96      0.13     18745

    accuracy                           0.62    655559
   macro avg       0.50      0.45      0.32    655559
weighted avg       0.75      0.62      0.62    655559



[{'0.0': {'precision': 1.0,
   'recall': 0.9825958555598492,
   'f1-score': 0.9912215369605742,
   'support': 463223.0},
  '1.0': {'precision': 0.5782692502789896,
   'recall': 0.12975483166017893,
   'f1-score': 0.21195106533298627,
   'support': 43929.0},
  '2.0': {'precision': 0.5043331059400187,
   'recall': 0.18394352958522628,
   'f1-score': 0.26956838807260153,
   'support': 108517.0},
  '3.0': {'precision': 0.3333333333333333,
   'recall': 6.593758536562391e-05,
   'f1-score': 0.00013184908929950463,
   'support': 106161.0},
  '4.0': {'precision': 0.06904893499426044,
   'recall': 0.9023780420046672,
   'f1-score': 0.12828188682822028,
   'support': 17998.0},
  'accuracy': 0.6718723811480506,
  'macro avg': {'precision': 0.49699692490932046,
   'recall': 0.43974763927905747,
   'f1-score': 0.32023094525673634,
   'support': 739828.0},
  'weighted avg': {'precision': 0.7839447118584664,
   'recall': 0.6718723811480506,
   'f1-score': 0.675890855392441,
   'support': 739828.0}}]

In [6]:
model = Seq2SeqTransformer(d_model=30, ff_dim=1024, num_heads=8, num_layers=6, num_classes=len(SAT_CLASSES_ACCURACY), emb_dim=512)
model.set_pretraining(False)
train_and_test(
    model,
    train_dataset,
    test_dataset,
    val_dataset,
    logs_path=Path("../logs/"),
    workers=4,
    batch_size=64,
    labels=SAT_CLASSES_ACCURACY,
    epochs=10,
    # epochs=1,
    weight_decay=0.001,
    label_smoothing=0.1,
    lr=0.00001
)

  0%|          | 0/167 [00:00<?, ? batch/s]

ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 ERROR: Unexpected bus error encountered in worker. This might be caused by insufficient shared memory (shm).
 

RuntimeError: DataLoader worker (pid(s) 166091, 166096, 166097, 166098, 166099) exited unexpectedly